<a href="https://colab.research.google.com/github/nodeblackbox/Faceted_judge_2022_08_29_03_31_59/blob/master/lab_5_BOSTON.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The Boston Housing Dataset (or California)

---

## 1. Practice

- Run k-fold validation on the Boston dataset;
- Notice that the mini-batch size is set to 1. Experiment with different mini-batch sizes. What do you observe? Can you account for your observation?
- Run a series of experiments to find the best model. (Hint: look back at the previous labs.)
- Retrain the best model on all the training data and evaluate on the test data.

#### Note

For those who wish to work on the California Housing Dataset instead, follow the instructions at the bottom of the notebook. This counts 'not covered' dataset in the first coursework.

In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

In [2]:
from tensorflow.keras.datasets import boston_housing
(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

57026/57026 [==============================] - 0s 0us/step


In [3]:
mean = train_data.mean(axis = 0)
train_data -= mean # shift
std = train_data.std(axis = 0)
train_data /= std # rescale

test_data -= mean
test_data /= std

In [4]:
from tensorflow.keras import models
from tensorflow.keras import layers

def build_model():
    model = models.Sequential()
    model.add(
        layers.Dense(
            64, activation = 'relu', input_shape = (train_data.shape[1], )
        )
    )
    model.add(layers.Dense(64, activation = 'relu'))
    model.add(layers.Dense(1))
    model.compile(
        optimizer='rmsprop',
        loss='mse',
        metrics=['mae']
    )
    
    return model

In [5]:
class CustomCallback(tf.keras.callbacks.Callback):
    def __init__(self, epochs):
        super(tf.keras.callbacks.Callback, self).__init__()
        self.epochs = epochs
    def on_epoch_begin(self, epoch, logs=None):
        c = ['|', '/', '-', '\\'] 
        print(f"\r{c[epoch % 4]} epoch: {epoch+1}/{self.epochs}", end="")
    def on_train_end(self, logs=None):
        print()

In [ ]:
K = 4
num_val_samples = len(train_data) // K
num_epochs = 500
all_mae_histories = []
for i in range(K):
    print('processing fold', i)
    
    # Prepare the validation data: data from partition i
    a, b = i * num_val_samples, (i + 1) * num_val_samples
    val_data = train_data[a : b]
    val_targets = train_targets[a : b]
    
    # Prepare the training data: data from all other partitions
    partial_train_data = np.concatenate([train_data[:a], train_data[b:]], axis=0)
    partial_train_targets = np.concatenate([train_targets[:a], train_targets[b:]], axis=0)

    # Build the Keras model (already compiled)
    model = build_model()
    
    # Train the model (in silent mode, verbose=0)
    history = model.fit(
        partial_train_data,
        partial_train_targets,
        validation_data=(val_data, val_targets),
        epochs=num_epochs, batch_size=1, verbose=0, 
        callbacks=[CustomCallback(num_epochs)]
    )

    mae_history = history.history['val_mae']
    all_mae_histories.append(mae_history)

processing fold 0
/ epoch: 242/500

In [ ]:
average_mae_history = np.array(all_mae_histories).mean(axis=0)
plt.plot(range(1, len(average_mae_history) + 1), average_mae_history)
plt.xlabel('Epochs')
plt.ylabel('Validation MAE')
plt.show()

In [ ]:
def smooth_curve(points, beta = 0.9):       # beta must be between 0 and 1!
    smoothed_points = []
    for current in points:
        if smoothed_points:                 # (an nonempty list is 'True')
            previous = smoothed_points[-1]  # the last appended point
                                            # ↓ a weighted sum of previous & point, controlled by beta
            smoothed_points.append(beta * previous + (1 - beta) * current)
        else:
            smoothed_points.append(current) # at the start, the list is empty, we just add the first point
    return smoothed_points

In [ ]:
smooth_mae_history = smooth_curve(average_mae_history[10:])
plt.plot(range(1, len(smooth_mae_history) + 1), smooth_mae_history)
plt.xlabel('Epochs')
plt.ylabel('Validation MAE')
plt.show()

---

# Optional

## The California Housing dataset

[California Housing](https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html), original website. (Also available on [Kaggle](https://www.kaggle.com/datasets/camnugent/california-housing-prices).)

### 1. Download

The terminal commands to download it. (Add a `!` in front of them to use them from Jupyter or Colab.)

```bash
wget https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.tgz
tar -xvf cal_housing.tgz
```

### 2. Load the data

Use the name of the file `cal_housing.data` to:
- open it 
- read the lines 
- strip the final newline `\n` 
- split on commas
- turn the data into a numpy array, casting it as floats

#### Note

You can see the features by loading `cal_housing.domain`, read its lines, and print its contents.

## 3. Separate the features and the targets

The price is the last feature, so you need to use NumPy to slice all the `targets` in the last dimension, and the rest of the `features` in another array.

## 4. Scale the prices to a more manageable range

You can print the `min()` and the `max()` of your `targets` to see the kind of range we are dealing with.

Then a division by `100000` will give us similar numbers to the Boston Housing Dataset.

Once you have your reduced targets, you may want to print again the `min()` and the `max()` as a sanity check.


## 5. Split your data into train and test

Use `.shape` on your `features` (and/or `targets`) to check how many samples this dataset has.

Slice both `features` and `targets` to obtain `train_data`, `test_data`, and `train_targets`, `test_targets` respectively.

This is actually a potential subject of experiment. You could slice it roughly in the middle, or have more in your training than your testing set.

As a sanity check, your shapes should look like this:
```
# n_train: number of training samples
# n_train: number of testing samples
train_data (n_train, 8)
train_targets (n_train,)
test_data (n_test, 8)
test_targets (n_test,) 
```

## 6. Normalisation/scaling

Use the mean and standard deviation of the **train data** to normalise it, and apply the same transform to test data, exactly as above with the Boston Housing Dataset.

## 7. Everything is now set up for training

The rest of the procedure (define the model, train, plot) is now the same.

#### Note

This dataset is not small like the Boston Housing Dataset, so you may find that it's taking too long to do many epochs with K-fold given the compute you have. This doesn't matter *too* much, the important thing is to understand the K-fold logic.

#### Experiment

You could artificially reduce your training set to a similar size as the Boston Housing Dataset (~400 samples), and see what performance you manage to get on your (then much, much larger) test set!

#### Visualisations

Three examples of how people use visualisations for this dataset:
- [California Housing Modelling and Map Visualisation](https://www.kaggle.com/code/qixuan/california-housing-modelling-and-map-visualisation)
- [California Housing Prices: EDA and Visualization](https://www.kaggle.com/code/ujwalkandi/california-housing-prices-eda-and-visualization)
- [The California housing dataset](https://inria.github.io/scikit-learn-mooc/python_scripts/datasets_california_housing.html)